## Deploy text-generation streaming models from HuggingFaceHub to AzureML Online Endpoints

This sample shows how to deploy `text-generation` streaming models from the HuggingFaceHub to an online endpoint for inference. Learn more about `text-generation` task: https://huggingface.co/tasks/text-generation

A large set of models hosted on [Hugging Face Hub](https://huggingface.co/models) are available in the Hugging Face Hub collection in AzureML Model Catalog. This collection is powered by the Hugging Face Hub community registry. Integration with the AzureML Model Catalog enables seamless deployment of Hugging Face Hub models in AzureML. [Learn more.](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-models-from-huggingface?view=azureml-api-2)
### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Deploy the model for real time inference.
* Try sample inference.
* Clean up resources.

### Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `HuggingFaceHub` community registry

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

# connect to a workspace
workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    workspace = workspace_ml_client.workspace_name
    resource_group = workspace_ml_client.resource_group_name
except Exception as ex:
    print(ex)
    # Enter details of your workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, workspace
    )
# Connect to the HuggingFaceHub registry
registry_ml_client = MLClient(credential, registry_name="HuggingFace")
print(registry_ml_client)

### Pick a model to deploy

Open the Model Catalog in AzureML Studio and choose the Hugging Face Hub collection. Filter by the `text-generation` task and search any specific models you are interested in. In this example, we use the `tiiuae-falcon-40b` model. If you plan to deploy a different model, replace the model name and version accordingly. 

In [ ]:
model_name = "tiiuae-falcon-40b"
foundation_model = registry_ml_client.models.get(model_name, version="5")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)

### Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model. Create an online endpoint and then create an online deployment. You need to specify the Virtual Machine instance or SKU when creating the deployment. You can find the optimal CPU or GPU SKU for a model by opening the quick deployment dialog from the model page in the AzureML Model Catalog. Specify the SKU in the `instance_type` input in deployment settings below.

Typically Online Endpoints require you to provide scoring script and a docker container image (through an AzureML environment), in addition to the model. You don't need to worry about them for HuggingFace Hub models available in AzureML Model Catalog because we have enabled 'no code deployments' for these models by packaging scoring script and container image along with the model.

Learn more about Online Endpoints: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints

In [ ]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "text-generation-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for text-generation task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_ND96amsr_A100_v4",
    instance_count=1,
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
# online endpoints can have multiple deployments with traffic split or shadow traffic. Set traffic to 100% for demo deployment
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### Try sample inference

Online endpoints expose a REST API that can be integrated into your applications. Learn how to fetch the scoring REST API and credentials for online endpoints here: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-authenticate-online-endpoint

In this example, we will use the Python SDK helper method to invoke the endpoint. 

In [ ]:
# Get the model object from HuggingFaceHub. We can use it to check for sample test data
import urllib.request, json

raw_data = urllib.request.urlopen(
    "https://huggingface.co/api/models/" + foundation_model.tags["modelId"]
)

print("https://huggingface.co/api/models/" + foundation_model.tags["modelId"])
data = json.load(raw_data)
print(data)

In [ ]:
# check if there is sample inference data available on HuggingFaceHub for the model, else try with the backup sample data
scoring_file = "./sample_score.json"
prompt = ""
if "widgetData" in data:
    for input in data["widgetData"]:
        prompt = input["text"]
        break
    # write the sample_score.json file
    score_dict = {"inputs": prompt}
    with open(scoring_file, "w") as outfile:
        json.dump(score_dict, outfile)
else:
    scoring_file = "./sample_score_backup.json"

# print the sample scoring file
print("\n\nSample scoring file: ")
with open(scoring_file) as json_file:
    scoring_data = json.load(json_file)
    print(scoring_data)

In [ ]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file=scoring_file,
)
response_json = json.loads(response)
print(json.dumps(response_json, indent=2))

### Stream responses from the online endpoint
Text-generation models can take considerable time to handle a request and return generated text. One way to improve the user experience is to stream tokens back to the user as they are generated. Learn more about streaming results from Hugging Face text-generation models here: https://huggingface.co/blog/inference-endpoints-llm.

In this example, we will use the Hugging Face python text-generation `InferencingClient` to help stream results from our online endpoint. Learn more about Hugging Face text generation inference here: https://github.com/huggingface/text-generation-inference

In [ ]:
# Get the online endpoint scoring uri
online_endpoint = workspace_ml_client.online_endpoints.get(online_endpoint_name)
online_endpoint_scoring_uri = online_endpoint.scoring_uri
print(online_endpoint_scoring_uri)

In [ ]:
# Get the online endpoint api key for authenticating our inferencing request
keys = workspace_ml_client.online_endpoints.get_keys(online_endpoint_name)
api_key = keys.primary_key
print(api_key)

In [ ]:
pip install text-generation

In [ ]:
pip install -U huggingface_hub

In [ ]:
pip install ipywidgets

In [ ]:
from huggingface_hub import InferenceClient

# HF Inference Endpoints parameter
# Note - do not use the full /generate scoring url here
endpoint_url = online_endpoint_scoring_uri.split("/generate")[0]
print("Online endpoint url: {0}".format(endpoint_url))
headers = {"Content-Type": "application/json"}

# Streaming Client
client = InferenceClient(endpoint_url, token=api_key, headers=headers)

# generation parameter
gen_kwargs = dict(
    max_new_tokens=512,
    top_k=30,
    top_p=0.9,
    temperature=0.2,
    repetition_penalty=1.02,
    stop_sequences=["\nUser:", "<|endoftext|>", "</s>"],
)

# Begin text generation stream
print("Prompt:\n{0}".format(prompt))
stream = client.text_generation(prompt, stream=True, details=True, **gen_kwargs)
print("Stream result:\n")
# yield each generated token
for r in stream:
    # skip special tokens
    if r.token.special:
        continue
    # stop if we encounter a stop sequence
    if r.token.text in gen_kwargs["stop_sequences"]:
        break
    # yield the generated token
    print(r.token.text, end="")
    # yield r.token.text

### Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint.

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()